# Power System Analysis 2025/26 @ IST Work assessment


In [2]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

First, we read the excel file with the network data and create a dictionary of datasets, one for each sheet. As we can see from the output of dfs.keys(), we now have five datasets: 'Nodes', 'Transformers', 'Generators', 'Lines' and 'Loads'.

In [3]:
path = "network_data.xlsx"
xls = pd.ExcelFile(path)

# Dictionary to hold DataFrames
dfs = {}

for sheet in xls.sheet_names:
    dfs[sheet] = pd.read_excel(xls, sheet_name=sheet)

# Display the keys to verify loading
dfs.keys()

dict_keys(['Nodes', 'Transformers', 'Generators', 'Lines', 'Loads'])

The next step is to properly define the base system, as well a the Fortescue matrix to use in later calculations. The base currents will be calculated based on the "Nodes" dataset, which presents the base voltages for each node of the system. Additionally, the base power is set to a fixed 100MVA.

Apart from this, the Resistances and Reactances were combined into a single Impedance collumn in all the datasets for easier handling - and converted from Ohm to per-unit when necessary.

In [4]:
# Define base system:

# Base system
Sb=100e6 # W (100 MVA)

# Compute base current
dfs["Nodes"]["Base Current [pu]"] = Sb/(math.sqrt(3)*dfs["Nodes"]["Base Voltage [V]"]) # A

# Transformation matrix
a=np.exp(2j*np.pi/3)
A= np.array([[1,1,1],[1,a**2,a],[1,a,a**2]])

# Maximum fault parameters
tf_max= 2 # seconds
Zf_max_ohm = 40 # Ohm
Zf_max_pu = Zf_max_ohm/(dfs["Nodes"]["Base Voltage [V]"]**2/Sb)

# Compute the network impedances in pu

# Convert to the correct base 
dfs["Transformers"]["Z [pu]"] = (
    (dfs["Transformers"]["R [pu]"] + 1j * dfs["Transformers"]["X [pu]"])
    * (Sb / (dfs["Transformers"]["Power [MVA]"] * 1e6))
)

dfs["Generators"]["Z [pu]"] = (
    (dfs["Generators"]["R [pu]"] + 1j * dfs["Generators"]["X [pu]"])
    * (Sb / (dfs["Generators"]["Power [MVA]"] * 1e6))
)


dfs["Lines"]["Z1 [ohm]"] = dfs["Lines"]["R1 [ohm]"] + dfs["Lines"]["X1 [ohm]"]*1j
dfs["Lines"]["Z0 [ohm]"] = dfs["Lines"]["R0 [ohm]"] + dfs["Lines"]["X0 [ohm]"]*1j

dfs["Lines"]["Z1 [pu]"] = dfs["Lines"]["Z1 [ohm]"]/(dfs["Lines"]["Voltage Level [V]"]**2/Sb)
dfs["Lines"]["Z0 [pu]"] = dfs["Lines"]["Z0 [ohm]"]/(dfs["Lines"]["Voltage Level [V]"]**2/Sb)

dfs["Nodes"].head()

,Node ID Number,Base Voltage [V],Base Current [pu]
0,1,150000,384.900179
1,2,150000,384.900179
2,3,150000,384.900179
3,4,150000,384.900179
4,5,150000,384.900179


We can now build our primitive Impedance and Admittance matrices. Since all the transformers in this system have Ynd windings, none of them outright blocks zero sequence current to flow into the Y winding. As such, they should all be included in the primitive zero sequence matrices, since they allow zero sequence current to flow to ground, and as such, represent an impedance in the zero sequence network.

Aditionally, no loads were included since load current is usually negligible when performing short-circuit analysis at the grid level.

In [5]:
# Primitive Impedance matrices
# Created by placing the impedances of each element in the diagonal of a matrix, ordered lines - generators - transformers

Zprim1 = np.diag(
    dfs["Lines"]["Z1 [pu]"].tolist()
    + dfs["Generators"]["Z [pu]"].tolist()
    + dfs["Transformers"]["Z [pu]"].tolist()
)

Zprim0 = np.diag(
    dfs["Lines"]["Z0 [pu]"].tolist()
    + dfs["Generators"]["Z [pu]"].tolist()
    + dfs["Transformers"]["Z [pu]"].tolist()
)

# Primitive admittance matrices
# Created by inverting the primitive impedance matrices, since [Y] = [Z]^-1

Yprim1 = np.diag(1 / np.diag(Zprim1))
Yprim0 = np.diag(1 / np.diag(Zprim0))


In [6]:
# Check Lines
line_Z = dfs["Lines"]["Z1 [pu]"].tolist()
for i, z in enumerate(line_Z):
    assert Zprim1[i, i] == z, f"Mismatch in line {i}"

# Check Generators
gen_Z = dfs["Generators"]["Z [pu]"].tolist()
for i, z in enumerate(gen_Z):
    idx = len(line_Z) + i
    assert Zprim1[idx, idx] == z, f"Mismatch in generator {i}"

# Check Transformers
trafo_Z = dfs["Transformers"]["Z [pu]"].tolist()
for i, z in enumerate(trafo_Z):
    idx = len(line_Z) + len(gen_Z) + i
    assert Zprim1[idx, idx] == z, f"Mismatch in transformer {i}"

print("All Zprim1 entries match C1 row order")

All Zprim1 entries match C1 row order


Now we must create the constrain matrices for the zero and positive sequence. These matrices will have a dimmension of 12x20 (number of nodes x number of branches). The main difference between the two sequences will be that, for the zero sequence, the generator will not be connected to the transformers (since it is connected via a delta winding).

Aditionally, I have defined the currents to always flow from the lowest number node to the highest number node.

Since the positive sequence network has the exact same topology as the actual network, we can simply grab the node connections in our datasets to build the constrain matrix:

In [7]:
# Construct the positive sequence constrain matrix C1
C1 = np.zeros((20, 12), dtype=int)  # 20 branches, 12 nodes

# Lines
for branch_idx, row in dfs["Lines"].iterrows():
    node_a = int(row["Node A"]) - 1  # 0-based indexing
    node_b = int(row["Node B"]) - 1

    C1[branch_idx, node_a] = -1
    C1[branch_idx, node_b] =  1

# Generators
start_row = len(dfs["Lines"])  # where generator rows begin
for gen_idx, row in dfs["Generators"].iterrows():
    node = int(row["Connecting Nodes"]) - 1
    row_idx = start_row + gen_idx
    C1[row_idx, node] = 1

# Transformers
start_row += len(dfs["Generators"])
for trafo_idx, row in dfs["Transformers"].iterrows():
    node_h = int(row["Node H"]) - 1
    node_x = int(row["Node X"]) - 1
    row_idx = start_row + trafo_idx
    C1[row_idx, node_h] = -1
    C1[row_idx, node_x] =  1

print(C1)


[[-1  1  0  0  0  0  0  0  0  0  0  0]
 [-1  0  0  1  0  0  0  0  0  0  0  0]
 [-1  0  0  0  1  0  0  0  0  0  0  0]
 [ 0 -1  1  0  0  0  0  0  0  0  0  0]
 [ 0 -1  0  1  0  0  0  0  0  0  0  0]
 [ 0 -1  0  0  1  0  0  0  0  0  0  0]
 [ 0 -1  0  0  0  1  0  0  0  0  0  0]
 [ 0  0 -1  0  1  0  0  0  0  0  0  0]
 [ 0  0 -1  0  0  1  0  0  0  0  0  0]
 [ 0  0  0 -1  1  0  0  0  0  0  0  0]
 [ 0  0  0  0 -1  1  0  0  0  0  0  0]
 [ 0  0 -1  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0 -1  0  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0 -1  1  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  1]
 [-1  0  0  0  0  0  0  0  0  1  0  0]
 [ 0 -1  0  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0 -1  0  0  0  0  1]]


Now for the negative sequence, we have to keep in mind that the transformers are connected to neutral instead of the node on the delta side, which for this case is always the node on collumn "Node X". Aditionally, to keep the convention set that current always flows from low node to high node, and assuming the neutral is node 0, I considered that the "Node H" column now inputted a 1 instead of a -1. We can construct the constrain matrix the same way:

In [8]:
# Construct the positive sequence constrain matrix C0
C0 = np.zeros((20, 12), dtype=int)  # 20 branches, 12 nodes

# Lines
for branch_idx, row in dfs["Lines"].iterrows():
    node_a = int(row["Node A"]) - 1  # 0-based indexing
    node_b = int(row["Node B"]) - 1

    C0[branch_idx, node_a] = -1
    C0[branch_idx, node_b] =  1

# Generators
start_row = len(dfs["Lines"])  # where generator rows begin
for gen_idx, row in dfs["Generators"].iterrows():
    node = int(row["Connecting Nodes"]) - 1
    row_idx = start_row + gen_idx
    C0[row_idx, node] = 1

# Transformers
start_row += len(dfs["Generators"])
for trafo_idx, row in dfs["Transformers"].iterrows():
    node_h = int(row["Node H"]) - 1
    row_idx = start_row + trafo_idx
    C0[row_idx, node_h] = 1


print(C0)

[[-1  1  0  0  0  0  0  0  0  0  0  0]
 [-1  0  0  1  0  0  0  0  0  0  0  0]
 [-1  0  0  0  1  0  0  0  0  0  0  0]
 [ 0 -1  1  0  0  0  0  0  0  0  0  0]
 [ 0 -1  0  1  0  0  0  0  0  0  0  0]
 [ 0 -1  0  0  1  0  0  0  0  0  0  0]
 [ 0 -1  0  0  0  1  0  0  0  0  0  0]
 [ 0  0 -1  0  1  0  0  0  0  0  0  0]
 [ 0  0 -1  0  0  1  0  0  0  0  0  0]
 [ 0  0  0 -1  1  0  0  0  0  0  0  0]
 [ 0  0  0  0 -1  1  0  0  0  0  0  0]
 [ 0  0 -1  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0 -1  0  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0 -1  1  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  1]
 [ 1  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  0  0  0  0  0]]


We now have everything needed to compute the node impedance and admittance matrices, using the following equations:
$$
[\overline{Y}] = [K]^{t}[\overline{y}][K]
$$

$$
[\overline{Z}]^{-1} = [\overline{Y}]
$$

In [9]:
# Positive Sequence Node Admittance Matrix Computation
Y1 = C1.T @ Yprim1 @ C1

# Zero Sequence Node Admittance Matrix Computation
Y0 = C0.T @ Yprim0 @ C0

# Node Impedance Matrices
Z1 = np.linalg.inv(Y1)
Z0 = np.linalg.inv(Y0)


## Three Phase Fault

For a three phase fault, we have equation to calculate the fault current at each node:
$$
\overline{I}_{1-\Delta k} = \frac{\overline{V}_{1 - pF}}{\overline{Z}_{1-kk}}
$$

Which, since $\overline{V}_{1 - pF}$ is always 1 in per-unit, becomes:

$$
\overline{I}_{1-\Delta k} = \overline{Z}_{1-kk}^{-1}
$$

Additionally, it's important to state that for a three phase fault, the current in all 3 phases (a, b, c) is the same and equal to the positive sequence current.

In [10]:
Zkk_TP = np.diag(Z1)              # self-impedances
If_pu_TP = 1 / Zkk_TP              # fault currents in pu

# Convert from pu to A and to 
If_A_TP = If_pu_TP * dfs["Nodes"]["Base Current [pu]"]

# Convert to RMS magnitude
I_rms = np.abs(If_A_TP) / np.sqrt(2)

# Convert to phase angle in degrees
I_deg = np.angle(If_A_TP, deg=True)

# Assume Tp (peak time) = 1 / 50Hz * 1000 ms for one cycle as placeholder
# Or you can set your actual Tp values
Tp_ms = np.full(I_rms.shape, 20.0)  # 50 Hz system -> 20 ms per cycle

# Combine into a DataFrame
df_fault = pd.DataFrame({
    "Irms[A]": I_rms,
    "Phase[deg]": I_deg,
    "Tp[ms]": Tp_ms
})



print(df_fault)

          Irms[A]  Phase[deg]  Tp[ms]
0    11788.874807  -88.557393    20.0
1     7503.513068  -86.882959    20.0
2     3811.124382  -84.475169    20.0
3     3386.384354  -83.850010    20.0
4     3031.226361  -84.111723    20.0
5     2809.664976  -84.814497    20.0
6     5933.812672  -87.511855    20.0
7      934.832680  -83.581849    20.0
8      695.571713  -84.248634    20.0
9   273501.964859  -89.455551    20.0
10  108890.328803  -88.579907    20.0
11  138384.913965  -89.255281    20.0


In [11]:
node = 0  # example node index
Zself = Z1[node, node]
print("Node self-impedance:", Zself)
I_fault = 1.0 / Zself  # pu assuming V_prefault = 1 pu
print("Fault current in pu:", I_fault)
I_fault_A = I_fault * dfs["Nodes"].loc[node, "Base Current [pu]"]
print("Fault current in A:", I_fault_A)

Node self-impedance: (0.000581219574228063+0.023079324122508613j)
Fault current in pu: (1.0904825287404507-43.30136293871313j)
Fault current in A: (419.726921009922-16666.702365962476j)


## Single Line to Ground Fault

$$\bar{I}_{0-\Delta k} = \bar{I}_{1-\Delta k} = \bar{I}_{2-\Delta k} = \frac{\bar{V}_{1-pF}}{\bar{Z}_{0-kk} + \bar{Z}_{1-kk} + \bar{Z}_{2-kk} + 3R_f} = \frac{\bar{V}_{1-pF}}{\bar{Z}_T}$$

In [14]:
# Fault current with Zf_max_pu
ZT_SLG = np.diag(Z1) + np.diag(Z1) + np.diag(Z0) + 3*Zf_max_pu
If_pu_SLG = 1.0 / ZT_SLG  # pu assuming V_prefault = 1 pu

# Fault current with zero fault impedance (bolted fault)
ZT_SLG_zero = np.diag(Z1) + np.diag(Z1) + np.diag(Z0)  # remove fault impedance
If_pu_SLG_zero = 1.0 / ZT_SLG_zero  # pu

# Base currents
I_base = dfs["Nodes"]["Base Current [pu]"].values  # A

# Symmetrical components transformation
a = np.exp(2j * np.pi / 3)
A = np.array([[1, 1, 1],
              [1, a**2, a],
              [1, a, a**2]])

# Stack I0, I1, I2 for each node
I_seq = np.vstack([If_pu_SLG, If_pu_SLG, If_pu_SLG])         # SLG with Zf
I_seq_zero = np.vstack([If_pu_SLG_zero, If_pu_SLG_zero, If_pu_SLG_zero])  # bolted fault

# Phase currents in pu
I_abc_pu = (A @ I_seq) / 3
I_abc_pu_zero = (A @ I_seq_zero) / 3

# Convert to Amperes
I_abc_A = I_abc_pu * I_base
I_abc_A_zero = I_abc_pu_zero * I_base

# Compute RMS and phase angle
I_abc_rms = np.abs(I_abc_A) / np.sqrt(2)
I_abc_rms_zero = np.abs(I_abc_A_zero) / np.sqrt(2)

I_abc_angle = np.angle(I_abc_A, deg=True)
I_abc_angle_zero = np.angle(I_abc_A_zero, deg=True)

# Build DataFrame
n_nodes = I_abc_A.shape[1]
df_SLG = pd.DataFrame({
    "Node": np.repeat(np.arange(n_nodes), 3),
    "Phase": np.tile(["A", "B", "C"], n_nodes),
    "Irms[A]": I_abc_rms.T.flatten(),
    "Phase[deg]": I_abc_angle.T.flatten(),
    "Irms_zero[A]": I_abc_rms_zero.T.flatten(),
    "Phase_zero[deg]": I_abc_angle_zero.T.flatten()
})

print(df_SLG)


    Node Phase       Irms[A]  Phase[deg]  Irms_zero[A]  Phase_zero[deg]
0      0     A  5.060544e+02   -6.070470  4.783716e+03       -88.519828
1      0     B  5.631390e-14  107.308253  5.862404e-13        14.121845
2      0     C  6.757473e-14   97.241046  6.403519e-13        20.468068
3      1     A  4.981631e+02   -9.758790  2.934557e+03       -86.848118
4      1     B  6.278654e-14   95.134722  3.687035e-13        19.436775
5      1     C  6.064935e-14   93.000906  3.024642e-13        20.187621
6      2     A  4.244490e+02  -27.885030  9.019386e+02       -83.638691
7      2     B  4.783048e-14   80.644985  1.145148e-13        22.738651
8      2     C  4.827383e-14   77.494549  1.196686e-13        24.889674
9      3     A  4.053363e+02  -30.996548  7.812746e+02       -83.035916
10     3     B  5.095828e-14   85.367061  7.867749e-14        26.096692
11     3     C  4.867010e-14   70.466951  9.649851e-14        21.153825
12     4     A  3.865653e+02  -34.770098  6.736255e+02       -83